## Data Exploration

In [1]:
from __future__ import division
import pymongo
import json
from sqlalchemy import create_engine

In [2]:
engine = create_engine('postgresql://lucka@localhost:5432/bugs')

### Check few lines

In [3]:
df_mini = pd.read_sql_query('select * from final limit 10',con=engine)
df_mini.head(2).T

,0,1
id,187257,187377
opening,2002-12-31 06:38:20,2003-01-01 14:49:54
reporter,17687,81611
current_status,resolved,resolved
current_resolution,fixed,duplicate
assigned_to_cnt,1,2
assigned_to_init,,font@layout.bugs
assigned_to_final,,blizzard@mozilla.org
bug_status_cnt,2,2
bug_status_init,new,unconfirmed


### Check Statistics

Load all data

In [4]:
df = pd.read_sql_query('select * from final',con=engine)

Sanity check

- Does `reports.current_resolution` correspond with the last `resolution.when`?
  - Yes, there are only 4 differences (all of them verified vs fixed, not a big deal)
- Does `reports.current_resolution` correspond with the last `bug_status.when`?
  - Mostly, there are 31 differences

In [5]:
df[df['resolution_final'] != df['current_resolution']]

,id,opening,reporter,current_status,current_resolution,assigned_to_cnt,assigned_to_init,assigned_to_final,bug_status_cnt,bug_status_init,...,severity_init,severity_final,short_desc_cnt,short_desc_init,short_desc_final,version_cnt,version_init,version_final,desc_init,closing
154284,6898,1999-05-21 10:10:06,3851,verified,fixed,1,,,5,new,...,major,major,1,ime bug-cannot move cursor to select the chara...,ime bug-cannot move cursor to select the chara...,1,trunk,trunk,Tested 5-21-09 Win32 build.\n\nStep of reprodu...,1999-07-07 06:37:43
195163,6429,1999-05-14 10:33:10,4346,verified,fixed,1,,,4,assigned,...,normal,normal,2,accountmanager.xul: non-localizable xul file,accountmanager.xul: non-localized xul file,1,trunk,trunk,"Please refer to the ""XUL Coding Style Guidelin...",1999-06-16 16:40:56
277316,7470,1999-06-02 09:00:08,4114,verified,fixed,6,ducarroz@netscape.com,tague@netscape.com,8,new,...,major,major,2,"[pp]linux only, backspace in compose area put ...","backspace in compose area put chinese word ""re...",1,trunk,trunk,Using 19990602008m7 build for linux on an HP V...,1999-06-08 08:07:27
318561,10359,1999-07-22 14:42:03,1689,verified,fixed,2,troy@netscape.com,dveditz@netscape.com,8,new,...,major,major,1,[8.3] raptor .dll names too long for win9x,[8.3] raptor .dll names too long for win9x,1,trunk,trunk,Raptor .DLL names need to be 8.3 on Win9x. Se...,1999-07-23 14:24:36


In [6]:
df[df['bug_status_final'] != df['current_status']]['id'].count()

31

Calculate how long it took for the bug to be closed

In [7]:
df['duration'] = df['closing'] - df['opening']

Look at overall statistics

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,327620,364544,218743,91,192327,350240,530426,825743
reporter,327620,155314,139350,1,20209,119750,281133,459470
assigned_to_cnt,327620,1.47772,0.841184,1,1,1,2,25
bug_status_cnt,327620,2.81466,1.29777,1,2,2,3,27
cc_cnt,327620,3.96505,6.07301,1,2,3,4,466
component_cnt,327620,1.28054,0.57975,1,1,1,1,13
op_sys_cnt,327620,1.11557,0.344739,1,1,1,1,7
priority_cnt,327620,1.10681,0.368071,1,1,1,1,9
product_cnt,327620,1.09318,0.334321,1,1,1,1,7
resolution_cnt,327620,2.19973,0.686749,1,2,2,2,20


### Reassignments

How much do bugs get reassigned to different priorities, components...?

In [9]:
total = len(df)
priority = sum(df['priority_init'] != df['priority_final'])
severity = sum(df['severity_init'] != df['severity_final'])
component = sum(df['component_init'] != df['component_final'])
product = sum(df['product_init'] != df['product_final'])

print '{} bugs in total'.format(total)
print ' {} ({:0.2f}%) bugs end up with different priority'.format(priority, priority*100/total)
print ' {} ({:0.2f}%) bugs end up with different severity'.format(severity, severity*100/total)
print ' {} ({:0.2f}%) bugs end up with different component'.format(component, component*100/total)
print ' {} ({:0.2f}%) bugs end up with different product'.format(product, product*100/total)

327620 bugs in total
 28565 (8.72%) bugs end up with different priority
 26527 (8.10%) bugs end up with different severity
 70131 (21.41%) bugs end up with different component
 25604 (7.82%) bugs end up with different product


### Unique Values (and Counts)

How many distinct values do we have for interesting attributes?

In [10]:
print '{} unique priorities'.format(df['priority_init'].nunique())
print '{} unique severities'.format(df['severity_init'].nunique())
print '{} unique components'.format(df['component_init'].nunique())
print '{} unique products'.format(df['product_init'].nunique())

7 unique priorities
8 unique severities
735 unique components
69 unique products


In [11]:
print 'values and counts for initial priority:'
print df['priority_init'].value_counts()

print 'values and counts for initial severity:'
print df['severity_init'].value_counts()

values and counts for initial priority:
      297838
--     24864
p3      4624
p2       216
p1        54
p5        14
p4        10
Name: priority_init, dtype: int64
values and counts for initial severity:
normal         222705
major           32778
critical        28759
enhancement     16673
minor           16436
trivial          6719
blocker          3549
                    1
Name: severity_init, dtype: int64


Components are out of the game - there are too many.

Products could still be doable. What if we keep only top few products? How does it change the reassignment rate?

In [12]:
df['product_init'].value_counts().head(10)

core                         157979
firefox                      114985
thunderbird                   30816
bugzilla                      13301
browser                        2191
webtools                       1761
psm                            1464
mozilla application suite       819
mailnews                        643
mozilla.org                     454
Name: product_init, dtype: int64

In [13]:
top_products = ['core', 'firefox', 'thunderbird', 'bugzilla', 'browser', 'webtools', 'psm']
df['top_product_init'] = df['product_init'].map(lambda x: x if x in top_products else 'other')
df['top_product_final'] = df['product_final'].map(lambda x: x if x in top_products else 'other')

top_product = sum(df['top_product_init'] != df['top_product_final'])

print '{} bugs in total'.format(total)
print ' {} ({:0.2f}%) bugs end up with different product'.format(top_product, top_product*100/total)

327620 bugs in total
 25563 (7.80%) bugs end up with different product


### Duplicates

How many duplicates are there?

In [14]:
print 'final resolution - note the amount of duplicates:'
print df['resolution_final'].value_counts()

final resolution - note the amount of duplicates:
fixed         114399
duplicate      90083
worksforme     48785
invalid        35370
incomplete     21711
wontfix        12309
expired         4945
moved             14
                   4
Name: resolution_final, dtype: int64
